In [1]:
import pandas as pd
import numpy as np
import os
import pickle
import matplotlib.pyplot as plt

In [2]:
df = pd.read_pickle(r'C:\Users\danys\Downloads\data_compressed.pkl')

In [3]:
df.shape

(5531451, 190)

In [4]:
df_columns = df.columns.to_list()
df_columns[-1]

'D_145'

In [5]:
cat_features = ['B_30', 'B_38', 'D_114', 'D_116', 'D_117', 'D_120', 'D_126', 'D_63', 'D_64', 'D_66', 'D_68']
num_features = [col for col in df if col not in cat_features + ['S_2', 'customer_ID', 'B_31', 'D_87']]

In [6]:
sampled_df = df.iloc[:15000, :]

In [7]:
aggregated_num = sampled_df.groupby('customer_ID')[num_features].agg(['mean', 'median', 'max', 'min', 'std'])

In [8]:
aggregated_cat = sampled_df.groupby('customer_ID')[cat_features].agg(['count', 'nunique'])

In [9]:
df_features = pd.concat([aggregated_cat, aggregated_num], axis=1)
df_features.reset_index(inplace=True)
df_features_sharp = df_features.drop('customer_ID', axis=1)
print(df_features_sharp.shape)

(1251, 897)


C:\Users\danys\AppData\Local\Temp/ipykernel_17768/4085544773.py:3: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  df_features_sharp = df_features.drop('customer_ID', axis=1)


In [10]:
df_features_sharp.head()

B_30          B_38         D_114         D_116         D_117          ...  \
  count nunique count nunique count nunique count nunique count nunique  ...   
0    13       1    13       1    13       1    13       1    13       1  ...   
1    13       1    13       1    13       1    13       1    13       1  ...   
2    13       1    13       1    13       2    13       1    13       1  ...   
3    13       1    13       1    13       1    13       1    13       2  ...   
4    13       1    13       2    13       1    13       1    13       1  ...   

      D_144                                             D_145            \
       mean    median       max       min       std      mean    median   
0  0.005283  0.005492  0.009616  0.000610  0.002598  0.005814  0.006362   
1  0.004218  0.003169  0.009568  0.000027  0.002871  0.004902  0.004469   
2  0.005113  0.004747  0.009415  0.000129  0.003638  0.004500  0.005110   
3  0.004768  0.004793  0.009919  0.000492  0.002654  0.005236  0.006356   
4  0.004380  0.004070  0.009436  0.000633  0.002633  0.004219  0.002884   

                                 
        max       min       std  
0  0.009827  0.000995  0.003294  
1  0.009390  0.000796  0.002729  
2  0.006932  0.000443  0.002152  
3  0.009836  0.000029  0.003406  
4  0.009666  0.000083  0.003426  

[5 rows x 897 columns]

In [73]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score, f1_score, classification_report
from sklearn.linear_model import SGDClassifier 

In [12]:
train_labels = pd.read_csv(r"C:\Users\danys\Downloads\train_new\train_labels.csv")

In [28]:
clf.classes_

array([0, 1], dtype=int64)

In [17]:
train_labels_sharp = train_labels.iloc[:1251]

In [42]:
clf = SGDClassifier(loss='modified_huber', class_weight= {0: 0.2, 1:0.8})

In [19]:
X = df_features_sharp
y = train_labels_sharp.target

In [20]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size= .30, random_state=42)
X_train, X_test = X_train.reset_index(), X_test.reset_index()  # that's done for the sake of starting from 0
X_train, X_test = X_train.drop('index', axis=1), X_test.drop('index', axis=1)

C:\Users\danys\AppData\Local\Temp/ipykernel_17768/2102111229.py:3: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  X_train, X_test = X_train.drop('index', axis=1), X_test.drop('index', axis=1)


In [21]:
for col in X_train.columns:
    if np.isnan(X_train[col]).sum().sum():
        X_train[col] = X_train[col].fillna(0)
for col in X_test.columns:
    if np.isnan(X_test[col]).sum().sum():
        X_test[col] = X_test[col].fillna(0)

In [43]:
clf.fit(X_train, y_train)

SGDClassifier(class_weight={0: 0.2, 1: 0.8}, loss='modified_huber')

In [44]:
predictions = clf.predict(X_test)

In [45]:
predictions_proba = clf.predict_proba(X_test)[:, clf.classes_[clf.classes_ == 1][0]]

In [53]:
scores = pd.DataFrame(
        {roc_auc_score(y_test, predictions_proba), f1_score(y_test, predictions_proba), clf.score(X_test, y_test)},
        index = ["roc_auc", "f1_score", "accuracy"], columns=['result']
)

In [54]:
scores

,result
roc_auc,0.832402
f1_score,0.730000
accuracy,0.856383


In [78]:
print(classification_report(y_test, predictions, target_names=target_names))

              precision    recall  f1-score   support

     class 0       0.93      0.88      0.90       283
     class 1       0.68      0.78      0.73        93

    accuracy                           0.86       376
   macro avg       0.80      0.83      0.82       376
weighted avg       0.87      0.86      0.86       376



In [55]:
from sklearn.ensemble import RandomForestClassifier

In [62]:
model = RandomForestClassifier(class_weight={0: 0.2, 1: 0.8}, oob_score=True)

In [63]:
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

In [64]:
y_pred_proba = model.predict_proba(X_test)[:, model.classes_[model.classes_ == 1][0]]

In [65]:
scores_rf = pd.DataFrame(
        {"scores" : [model.oob_score_, model.score(X_test, y_test), roc_auc_score(y_test, y_pred_proba), f1_score(y_test, y_pred)]},
        index = ["oob_score", "accuracy", "roc_auc", "f1_score"]
)

In [67]:
scores_rf

,scores
oob_score,0.864000
accuracy,0.875000
roc_auc,0.934971
f1_score,0.725146


In [75]:
target_names = ['class 0', 'class 1']

In [77]:
print(classification_report(y_test, y_pred, target_names=target_names))

              precision    recall  f1-score   support

     class 0       0.89      0.93      0.91       283
     class 1       0.76      0.66      0.71        93

    accuracy                           0.86       376
   macro avg       0.83      0.79      0.81       376
weighted avg       0.86      0.86      0.86       376



In [68]:
model_2 = RandomForestClassifier(class_weight={0: 0.2, 1: 1}, oob_score=True)

In [69]:
model_2.fit(X_train, y_train)
y_pred = model_2.predict(X_test)

In [70]:
y_pred_proba = model_2.predict_proba(X_test)[:, model_2.classes_[model_2.classes_ == 1][0]]

In [71]:
scores_rf_2 = pd.DataFrame(
        {"scores" : [model_2.oob_score_, model_2.score(X_test, y_test), roc_auc_score(y_test, y_pred_proba), f1_score(y_test, y_pred)]},
        index = ["oob_score", "accuracy", "roc_auc", "f1_score"]
)

In [72]:
scores_rf_2

,scores
oob_score,0.846857
accuracy,0.864362
roc_auc,0.934040
f1_score,0.705202
